In [3]:
# lat lon of 1st street jetty
# 36.83088925769251, -75.96727139259976


In [11]:
import requests

url = "https://marine-api.open-meteo.com/v1/marine?latitude=36.83088925769251&longitude=-75.96727139259976&hourly=swell_wave_height,swell_wave_period"
res = requests.get(url)
json_data = res.json()

In [17]:
hourly_data = json_data.get('hourly', {})
times = hourly_data.get('time', [])
heights = hourly_data.get('swell_wave_height', [])
periods = hourly_data.get('swell_wave_period', [])

In [20]:
periods

[8.4,
 8.25,
 8.2,
 8.15,
 8.15,
 8.15,
 8.15,
 8.2,
 8.25,
 8.3,
 8.35,
 8.35,
 8.4,
 8.5,
 8.55,
 8.65,
 8.75,
 8.85,
 8.75,
 8.2,
 7.45,
 6.85,
 6.65,
 6.65,
 6.7,
 6.75,
 6.8,
 6.85,
 6.8,
 6.7,
 6.6,
 6.55,
 6.55,
 6.6,
 6.7,
 6.85,
 7.0,
 7.25,
 7.55,
 7.75,
 7.8,
 7.8,
 7.85,
 7.95,
 8.05,
 8.2,
 8.5,
 8.85,
 9.1,
 9.15,
 9.15,
 9.15,
 9.25,
 9.4,
 9.35,
 9.0,
 8.5,
 7.95,
 7.4,
 6.8,
 6.45,
 6.4,
 6.6,
 6.7,
 6.65,
 6.6,
 6.55,
 6.6,
 6.65,
 6.7,
 6.65,
 6.5,
 6.45,
 6.45,
 6.55,
 6.6,
 6.65,
 6.75,
 6.75,
 6.7,
 6.6,
 6.5,
 6.35,
 6.15,
 6.05,
 6.15,
 6.4,
 6.5,
 6.45,
 6.25,
 6.1,
 5.9,
 5.75,
 5.6,
 5.55,
 5.55,
 5.55,
 5.55,
 5.55,
 5.6,
 5.7,
 5.8,
 5.85,
 5.8,
 5.7,
 5.65,
 5.75,
 5.9,
 6.1,
 6.3,
 6.55,
 6.6,
 6.45,
 6.15,
 5.8,
 5.35,
 4.85,
 4.5,
 4.5,
 4.65,
 4.8,
 4.85,
 4.85,
 4.8,
 4.7,
 4.55,
 4.35,
 4.05,
 3.7,
 3.5,
 3.45,
 3.55,
 3.65,
 3.7,
 3.8,
 3.85,
 3.85,
 3.8,
 3.75,
 3.7,
 3.6,
 3.55,
 3.55,
 3.6,
 3.6,
 3.6,
 3.55,
 3.55,
 3.55,
 3.6,
 3.65,
 3.7,
 3.7

In [21]:
from backend.app.models.models import SwellData

In [23]:
def parse_swell_data(json_data):
    hourly_data = json_data.get('hourly', {})
    times = hourly_data.get('time', [])
    heights = hourly_data.get('swell_wave_height', [])
    periods = hourly_data.get('swell_wave_period', [])

    surf_data_list = []

    for time, height, period in zip(times, heights, periods):
        surf_data = SwellData(
            latitude=json_data.get('latitude'),
            longitude=json_data.get('longitude'),
            generationtime_ms=json_data.get('generationtime_ms'),
            utc_offset_seconds=json_data.get('utc_offset_seconds'),
            timezone=json_data.get('timezone'),
            timezone_abbreviation=json_data.get('timezone_abbreviation'),
            elevation=json_data.get('elevation'),
            hourly_units=json_data.get('hourly_units'),
            hourly=time,
            swell_wave_height=height,
            swell_wave_period=period
        )
        surf_data_list.append(surf_data)

    return surf_data_list

In [30]:
import sys
sys.path.append('./backend/app/')

In [37]:
from models.models import SwellData
from db.database import get_db
from sqlalchemy.ext.asyncio import AsyncSession
from contextlib import asynccontextmanager


@asynccontextmanager
async def get_db():
    async with AsyncSessionLocal() as session:
        yield session
        


In [38]:
async def save_swell_data(parsed_data):
    async with get_db() as session:  # This now correctly provides an async context manager
        for data in parsed_data:
            session.add(data)  # Add each data instance to the session
        await session.commit()  # Commit the transaction

In [40]:
parsed_swell_data = parse_swell_data(json_data)

In [42]:
save_swell_data(parsed_swell_data)

<coroutine object save_swell_data at 0x7f3ad5959d90>